# Physionet 2017 | ECG Rhythm Classification
## 4. Train Model
### Sebastian D. Goodfellow, Ph.D.

# Setup Noteboook

In [6]:
# Import 3rd party libraries
import os
import sys
import numpy as np
import pickle

# Deep learning libraries
import tensorflow as tf

# Import local Libraries
sys.path.insert(0, r'C:\Users\sebastian goodfellow\Documents\code\deeP_ecg')
from deepecg.training.utils.plotting.time_series import plot_time_series_widget
from deepecg.training.utils.devices.device_check import print_device_counts
# from deepecg.training.train.memory.train import train
# from deepecg.training.model.memory.model import Model
from deepecg.config.config import DATA_DIR

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Resources

In [3]:
# Objective Function
# https://stackoverflow.com/questions/44560549/unbalanced-data-and-weighted-cross-entropy

# Global Average Pooling
# https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/
# https://github.com/philipperemy/tensorflow-class-activation-mapping/blob/master/class_activation_map.py
# https://github.com/AndersonJo/global-average-pooling

# 1. Load ECG Dataset

In [9]:
# Set path
path = os.path.join(DATA_DIR, 'training', 'memory')

# Set sample rate
fs = 300

print(os.path.join(path, 'training_60s.pickle'))

# Unpickle
with open(os.path.join(path, 'training_60s.pickle'), "rb") as input_file:
    data = pickle.load(input_file)

C:\Users\sebastian goodfellow\Documents\code\deeP_ecg\data\training\memory\training_60s.pickle


EOFError: Ran out of input

In [5]:
# Get training data
x_train = data['data_train'].values.reshape(data['data_train'].shape[0], data['data_train'].shape[1], 1)
y_train = data['labels_train']['label_int'].values.reshape(data['labels_train'].shape[0], 1).astype(int)

# Get validation data
x_val = data['data_val'].values.reshape(data['data_val'].shape[0], data['data_val'].shape[1], 1)
y_val = data['labels_val']['label_int'].values.reshape(data['labels_val'].shape[0], 1).astype(int)

# Print dimensions
print('x_train dimensions: ' + str(x_train.shape))
print('y_train dimensions: ' + str(y_train.shape))
print('x_val dimensions: ' + str(x_val.shape))
print('y_val dimensions: ' + str(y_val.shape))

x_train dimensions: (5774, 18000, 1)
y_train dimensions: (5774, 1)
x_val dimensions: (2475, 18000, 1)
y_val dimensions: (2475, 1)


In [6]:
# One hot encoding array dimensions
y_train_1hot = one_hot_encoding(labels=y_train.ravel(), classes=len(np.unique(y_train.ravel())))
y_val_1hot = one_hot_encoding(labels=y_val.ravel(), classes=len(np.unique(y_val.ravel())))

# Print dimensions
print('x_train dimensions: ' + str(x_train.shape))
print('y_train dimensions: ' + str(y_train.shape))
print('y_train_1hot dimensions: ' + str(y_train_1hot.shape))
print('x_val dimensions: ' + str(x_val.shape))
print('y_val dimensions: ' + str(y_val.shape))
print('y_val_1hot dimensions: ' + str(y_val_1hot.shape))

x_train dimensions: (5774, 18000, 1)
y_train dimensions: (5774, 1)
y_train_1hot dimensions: (5774, 3)
x_val dimensions: (2475, 18000, 1)
y_val dimensions: (2475, 1)
y_val_1hot dimensions: (2475, 3)


In [7]:
# Label lookup
label_lookup = {'N': 0, 'A': 1, 'O': 2, '~': 3}

# Label dimensions
print('Train: Classes: ' + str(np.unique(y_train.ravel())))
print('Train: Count: ' + str(np.bincount(y_train.ravel())))
print('Val: Classes: ' + str(np.unique(y_val.ravel())))
print('Val: Count: ' + str(np.bincount(y_val.ravel())))

Train: Classes: [0 1 2]
Train: Count: [3553  531 1690]
Val: Classes: [0 1 2]
Val: Count: [1523  227  725]


In [8]:
# Label dictionary
label_list = ['Normal Sinus Rhythm', 'Atrial Fibrillation', 'Other Rhythm']

# PLot times series
plot_time_series_widget(time_series=x_train, labels=y_train, fs=fs, label_list=label_list)

interactive(children=(IntSlider(value=2886, description='index', max=5773), Output()), _dom_classes=('widget-i…

# 2. Device Check

In [9]:
# Get GPU count
print_device_counts()

Workstation has 1 CPUs.
Workstation has 2 GPUs.


# 3. Initialize Model

In [15]:
# Set save path for graphs, summaries, and checkpoints
save_path = r'C:\Users\sebastian goodfellow\Desktop\tensorboard\deep_ecg'

# Set model name
model_name = 'test_1'

# Maximum number of checkpoints to keep
max_to_keep = 20

# Set randome states
seed = 0                                    
tf.set_random_seed(seed)                      

# Get training dataset dimensions
(m, length, channels) = x_train.shape  

# Get number of label classes
classes = y_train_1hot.shape[1]                  

# Choose network
network_name = 'DeepECG'

# Set network inputs
network_parameters = dict(
    length=length,
    channels=channels, 
    classes=classes, 
    seed=seed,  
)

# Create model
model = Model(
    model_name=model_name, 
    network_name=network_name, 
    network_parameters=network_parameters, 
    save_path=save_path,
    max_to_keep=max_to_keep
)

CPU Assign
/gpu:0
CPU Assign
/gpu:1


InvalidArgumentError: Malformed device specification '//cpu:0'
	 [[Node: train_op/DeepECG/logits/kernel/Adam_1 = VariableV2[_class=["loc:@DeepECG/logits/kernel"], container="", dtype=DT_FLOAT, shape=[64,3], shared_name="", _device="//cpu:0"]()]]

Caused by op 'train_op/DeepECG/logits/kernel/Adam_1', defined at:
  File "C:\Continuum\anaconda3\envs\deep_ecg\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Continuum\anaconda3\envs\deep_ecg\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Continuum\anaconda3\envs\deep_ecg\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Continuum\anaconda3\envs\deep_ecg\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Continuum\anaconda3\envs\deep_ecg\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Continuum\anaconda3\envs\deep_ecg\l

# 7. Train Model

In [ ]:
# Set hyper-parameters
epochs = 100
minibatch_size =  10
learning_rate = 0.001            

# Train model
train(
    model=model, 
    x_train=x_train, y_train=y_train_1hot, 
    x_val=x_val, y_val=y_val_1hot,
    learning_rate=learning_rate,
    epochs=epochs, mini_batch_size=minibatch_size, 
)